In [1]:
import seaborn as sns
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

In [2]:
cencus = pd.read_csv("Family Income and Expenditure.csv")

In [3]:
cencus.head(5)

,Total Household Income,Region,Total Food Expenditure,Main Source of Income,Agricultural Household indicator,Bread and Cereals Expenditure,Total Rice Expenditure,Meat Expenditure,Total Fish and marine products Expenditure,Fruit Expenditure,...,Number of Refrigerator/Freezer,Number of Washing Machine,Number of Airconditioner,"Number of Car, Jeep, Van",Number of Landline/wireless telephones,Number of Cellular phone,Number of Personal Computer,Number of Stove with Oven/Gas Range,Number of Motorized Banca,Number of Motorcycle/Tricycle
0,480332,CAR,117848,Wage/Salaries,0,42140,38300,24676,16806,3325,...,1,1,0,0,0,2,1,0,0,1
1,198235,CAR,67766,Wage/Salaries,0,17329,13008,17434,11073,2035,...,0,1,0,0,0,3,1,0,0,2
2,82785,CAR,61609,Wage/Salaries,1,34182,32001,7783,2590,1730,...,0,0,0,0,0,0,0,0,0,0
3,107589,CAR,78189,Wage/Salaries,0,34030,28659,10914,10812,690,...,0,0,0,0,0,1,0,0,0,0
4,189322,CAR,94625,Wage/Salaries,0,34820,30167,18391,11309,1395,...,1,0,0,0,0,3,0,0,0,1


In [4]:
cencus['Region'].value_counts()

Region
IVA - CALABARZON             4162
NCR                          4130
III - Central Luzon          3237
VI - Western Visayas         2851
VII - Central Visayas        2541
V - Bicol Region             2472
XI - Davao Region            2446
I - Ilocos Region            2348
VIII - Eastern Visayas       2337
 ARMM                        2248
II - Cagayan Valley          2219
XII - SOCCSKSARGEN           2122
X - Northern Mindanao        1887
IX - Zasmboanga Peninsula    1788
Caraga                       1782
CAR                          1725
IVB - MIMAROPA               1249
Name: count, dtype: int64

In [5]:
cencus['Main Source of Income'].value_counts()

Main Source of Income
Wage/Salaries                 20388
Other sources of Income       10836
Enterpreneurial Activities    10320
Name: count, dtype: int64

In [6]:
cencus['Bread and Cereals Expenditure'].value_counts()

Bread and Cereals Expenditure
0        15
20734     9
8758      8
15520     8
21930     8
         ..
24887     1
8196      1
16396     1
11196     1
16430     1
Name: count, Length: 26082, dtype: int64

In [7]:
cencus['Total Rice Expenditure'].value_counts()

Total Rice Expenditure
0        715
11700     95
14480     94
15600     92
10400     89
        ... 
25190      1
36312      1
43239      1
39217      1
38213      1
Name: count, Length: 16145, dtype: int64

In [8]:
cencus['Meat Expenditure'].value_counts()

Meat Expenditure
0        346
900       41
1620      39
1740      38
960       33
        ... 
1763       1
4246       1
2779       1
8693       1
16123      1
Name: count, Length: 18619, dtype: int64

In [9]:
cencus['Transportation Expenditure'].value_counts()

Transportation Expenditure
0        309
1200     266
2400     232
1800     222
960      217
        ... 
27348      1
19734      1
37176      1
24426      1
91500      1
Name: count, Length: 7435, dtype: int64

In [10]:
cencus['Total Income from Entrepreneurial Acitivites']

0         44370
1             0
2             0
3         15580
4         75687
          ...  
41539     94120
41540     78270
41541     98545
41542    107435
41543    105547
Name: Total Income from Entrepreneurial Acitivites, Length: 41544, dtype: int64

In [11]:
cencus['Household Head Sex'].value_counts()

Household Head Sex
Male      32483
Female     9061
Name: count, dtype: int64

In [12]:
cencus['Household Head Age'].value_counts()

Household Head Age
53    1146
46    1140
45    1133
49    1123
50    1123
      ... 
97       3
13       2
10       1
14       1
9        1
Name: count, Length: 89, dtype: int64

In [13]:
cencus['Household Head Marital Status'].value_counts()

Household Head Marital Status
Married               31347
Widowed                6818
Single                 1942
Divorced/Separated     1425
Annulled                 11
Unknown                   1
Name: count, dtype: int64

In [14]:
cencus['Household Head Highest Grade Completed'].value_counts()

Household Head Highest Grade Completed
High School Graduate                                                                                                                                                                                                   9628
Elementary Graduate                                                                                                                                                                                                    7640
Grade 4                                                                                                                                                                                                                2282
Grade 5                                                                                                                                                                                                                2123
Second Year High School                                                          

In [15]:
cencus['Household Head Job or Business Indicator'].value_counts()

Household Head Job or Business Indicator
With Job/Business    34008
No Job/Business       7536
Name: count, dtype: int64

In [16]:
cencus['Household Head Occupation'].value_counts()

Household Head Occupation
Farmhands and laborers                                                                 3478
Rice farmers                                                                           2849
General managers/managing proprietors in wholesale and retail trade                    2028
General managers/managing proprietors in transportation, storage and communications    1932
Corn farmers                                                                           1724
                                                                                       ... 
Coding, proof-reading and related clerks                                                  1
Advertising and public relations managers                                                 1
Wood treaters                                                                             1
Aircraft pilots, navigators and flight engineers                                          1
Glass, ceramics and related plant operators n. e. c.  

In [17]:
cencus['Household Head Class of Worker'].value_counts()

Household Head Class of Worker
Self-employed wihout any employee                             13766
Worked for private establishment                              13731
Worked for government/government corporation                   2820
Employer in own family-operated farm or business               2581
Worked for private household                                    811
Worked without pay in own family-operated farm or business      285
Worked with pay in own family-operated farm or business          14
Name: count, dtype: int64

In [18]:
cencus['Type of Household'].value_counts()

Type of Household
Single Family                             28445
Extended Family                           12932
Two or More Nonrelated Persons/Members      167
Name: count, dtype: int64

In [19]:
cencus['Total Number of Family members'].value_counts()

Total Number of Family members
4     7921
5     7165
3     6509
6     5222
2     4659
7     3222
1     2511
8     1984
9     1063
10     621
11     311
12     156
13      80
14      51
15      36
16      14
17       7
18       5
20       4
19       2
26       1
Name: count, dtype: int64

In [20]:
cencus['Members with age less than 5 year old'].value_counts()

Members with age less than 5 year old
0    28705
1     9317
2     2933
3      511
4       64
5       14
Name: count, dtype: int64

In [21]:
cencus['Members with age 5 - 17 years old'].value_counts()

Members with age 5 - 17 years old
0    14802
1    10445
2     8111
3     4704
4     2152
5      896
6      318
7       96
8       20
Name: count, dtype: int64

In [22]:
cencus['Total number of family members employed'].value_counts()

Total number of family members employed
1    15312
0    11494
2     9303
3     3579
4     1280
5      415
6      116
7       33
8       12
Name: count, dtype: int64

In [23]:
cencus['Type of Building/House'].value_counts()

Type of Building/House
Single house                                   39069
Multi-unit residential                          1329
Duplex                                          1084
Commercial/industrial/agricultural building       51
Institutional living quarter                       9
Other building unit (e.g. cave, boat)              2
Name: count, dtype: int64

In [24]:
cencus['Type of Roof'].value_counts()

Type of Roof
Strong material(galvanized,iron,al,tile,concrete,brick,stone,asbestos)    33342
Light material (cogon,nipa,anahaw)                                         5074
Mixed but predominantly strong materials                                   2002
Mixed but predominantly light materials                                     846
Salvaged/makeshift materials                                                212
Mixed but predominantly salvaged materials                                   56
Not Applicable                                                               12
Name: count, dtype: int64

In [25]:
cencus['Type of Walls'].value_counts()

Type of Walls
Strong            27739
Light              8267
Quite Strong       3487
Very Light         1583
Salvaged            456
NOt applicable       12
Name: count, dtype: int64

In [26]:
cencus['House Floor Area'].value_counts()

House Floor Area
30     2976
20     2903
40     2643
50     2237
80     1838
       ... 
279       1
275       1
226       1
254       1
151       1
Name: count, Length: 313, dtype: int64

In [27]:
cencus['House Age'].value_counts()

House Age
15     3382
25     2206
20     2072
35     1559
5      1553
       ... 
105       1
200       1
91        1
94        1
132       1
Name: count, Length: 111, dtype: int64

In [28]:
cencus['Number of bedrooms'].value_counts()

Number of bedrooms
2    15456
1    13431
3     6111
0     3930
4     1875
5      484
6      169
7       46
8       29
9       13
Name: count, dtype: int64

In [29]:
cencus['Tenure Status'].value_counts()

Tenure Status
Own or owner-like possession of house and lot        29541
Own house, rent-free lot with consent of owner        6165
Rent house/room including lot                         2203
Rent-free house and lot with consent of owner         2014
Own house, rent-free lot without consent of owner      995
Own house, rent lot                                    425
Rent-free house and lot without consent of owner       128
Not Applicable                                          73
Name: count, dtype: int64

In [30]:
cencus['Toilet Facilities'].value_counts()

Toilet Facilities
Water-sealed, sewer septic tank, used exclusively by household    29162
Water-sealed, sewer septic tank, shared with other household       3694
Water-sealed, other depository, used exclusively by household      2343
Closed pit                                                         2273
Open pit                                                           1189
Water-sealed, other depository, shared with other household         950
Others                                                              353
Name: count, dtype: int64

In [31]:
cencus['Electricity'].value_counts()

Electricity
1    37008
0     4536
Name: count, dtype: int64

In [32]:
cencus['Main Source of Water Supply'].value_counts()

Main Source of Water Supply
Own use, faucet, community water system    16093
Shared, tubed/piped deep well               6242
Shared, faucet, community water system      4614
Own use, tubed/piped deep well              4587
Dug well                                    3876
Protected spring, river, stream, etc        2657
Tubed/piped shallow well                    1394
Peddler                                      851
Unprotected spring, river, stream, etc       607
Lake, river, rain and others                 496
Others                                       127
Name: count, dtype: int64

In [33]:
cencus['Number of Television'].value_counts()

Number of Television
1    27089
0    10717
2     2955
3      597
4      133
5       42
6       11
Name: count, dtype: int64

In [34]:
cencus['Number of CD/VCD/DVD'].value_counts()

Number of CD/VCD/DVD
0    24621
1    15983
2      752
3      163
4       20
5        5
Name: count, dtype: int64

In [35]:
cencus['Number of Component/Stereo set'].value_counts()

Number of Component/Stereo set
0    35058
1     6284
2      174
3       13
4       10
5        5
Name: count, dtype: int64

In [36]:
cencus['Number of Refrigerator/Freezer'].value_counts()

Number of Refrigerator/Freezer
0    25990
1    14881
2      569
3       73
4       17
5       14
Name: count, dtype: int64

In [37]:
cencus['Number of Washing Machine'].value_counts()

Number of Washing Machine
0    28484
1    12845
2      204
3       11
Name: count, dtype: int64

In [38]:
cencus['Number of Washing Machine'].value_counts()

Number of Washing Machine
0    28484
1    12845
2      204
3       11
Name: count, dtype: int64

In [39]:
cencus['Number of Airconditioner'].value_counts()

Number of Airconditioner
0    37457
1     3178
2      622
3      199
4       66
5       22
Name: count, dtype: int64

In [40]:
cencus['Number of Car, Jeep, Van'].value_counts()

Number of Car, Jeep, Van
0    38876
1     2136
2      413
3       77
4       29
5       13
Name: count, dtype: int64

In [41]:
cencus['Number of Landline/wireless telephones'].value_counts()

Number of Landline/wireless telephones
0    39302
1     2070
2       96
3       48
4       28
Name: count, dtype: int64

In [42]:
cencus['Number of Cellular phone'].value_counts()

Number of Cellular phone
1     12484
2     10377
0      6939
3      5820
4      3281
5      1467
6       666
7       242
8       153
10       66
9        49
Name: count, dtype: int64

In [43]:
cencus['Number of Personal Computer'].value_counts()

Number of Personal Computer
0    32988
1     5650
2     1836
3      667
4      271
5      112
6       20
Name: count, dtype: int64

In [44]:
cencus['Number of Stove with Oven/Gas Range'].value_counts()

Number of Stove with Oven/Gas Range
0    36101
1     5287
2      145
3       11
Name: count, dtype: int64

In [45]:
cencus['Number of Motorized Banca'].value_counts()

Number of Motorized Banca
0    41055
1      444
2       34
3       11
Name: count, dtype: int64

In [46]:
cencus['Number of Motorcycle/Tricycle'].value_counts()

Number of Motorcycle/Tricycle
0    31282
1     8811
2     1199
3      186
4       54
5       12
Name: count, dtype: int64

<h1>Exploratory Data Analysis<h1>

In [47]:
census = pd.read_csv("Family Income and Expenditure.csv")

In [48]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

cencus.dtypes

Total Household Income                            int64
Region                                           object
Total Food Expenditure                            int64
Main Source of Income                            object
Agricultural Household indicator                  int64
Bread and Cereals Expenditure                     int64
Total Rice Expenditure                            int64
Meat Expenditure                                  int64
Total Fish and  marine products Expenditure       int64
Fruit Expenditure                                 int64
Vegetables Expenditure                            int64
Restaurant and hotels Expenditure                 int64
Alcoholic Beverages Expenditure                   int64
Tobacco Expenditure                               int64
Clothing, Footwear and Other Wear Expenditure     int64
Housing and water Expenditure                     int64
Imputed House Rental Value                        int64
Medical Care Expenditure                        

<h2>1. Regional Cost of Living: Where Survival Is Hardest</h2>


In [ ]:
#jp

<h2>2. The Regional “Saving Gap”: Which Regions Can Actually Save?</h2>


<h2>3. Are Rural or Urban Households More Financially Vulnerable?</h2>


<h2>4. The Hidden Poverty Behind the Walls</h2>


In [ ]:
#jp

<h2>5. “Imputed Rental” Inequality</h2>


<h2>6. Toilet & Water Access as a Poverty Indicator</h2>


<h2>7. Large Families, Small Budgets</h2>


<h2>8. Does Education Suffer with More Kids?</h2>


In [ ]:
#jp

<h2>9. Child-Heavy Households and Poverty Correlation</h2>


<h2>10. Who Are the Working Poor?</h2>


<h2>11. Entrepreneurial Families: Are They Better Off?</h2>


<h2>12. Occupation vs. Income Ranking</h2>


<h2>13. Food vs. Vices</h2>


<h2>14. Education vs. Entertainment Spending</h2>


In [ ]:
#jp

<h2>15. Medical vs. Special Occasions: What Hurts Families Most?</h2>


In [ ]:
#jp

<h2>16. Digital Divide Across Regions</h2>


<h2>17. Are Airconditioners and Cars a Sign of Wealth?</h2>


In [ ]:
#jp

<h2>18. Appliance Ownership vs. Basic Needs</h2>


<h2>19. Does the Head’s Education Affect Household Income?</h2>


<h2>20. Are Educated Heads Better at Budgeting?</h2>


<h2>21. The “Invisible Rich”: Low Income, High Assets</h2>


<h2>22. High Expenditure, Low Income = Debt?</h2>


<h2>23. Senior-Headed Households: Are They Poorer?</h2>


<h2>24. Gender and Poverty</h2>


In [ ]:
#jp

<h2>25. Who Spends More on Transportation?</h2>
